In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path 
from matplotlib.path import Path
import girder_client
import numpy as np
from matplotlib import pylab as plt
from matplotlib.colors import ListedColormap
import girder_client
import matplotlib.tri as T
import histomicstk as htk
from skimage.morphology import erosion, disk, opening, remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.measure import label
from skimage import segmentation

import pandas as pd
from histomicstk.saliency.tissue_detection import (
    get_slide_thumbnail, get_tissue_mask)

%matplotlib inline

In [ ]:
env_path= os.path.join('..', ".env")
load_dotenv(dotenv_path=env_path)

APIURL = os.getenv('APIURL')
APIKEY = os.getenv('APIKEY')
SAMPLE_SLIDE_ID = '5e4af18fe2ba484e10a75108' 
#SAMPLE_SLIDE_ID = "5e4af108e2ba484e10a750f3"
#SAMPLE_SLIDE_ID = '5fca5ee089f158714d9a9385'


In [ ]:
def get_connection(APIURL, APIKEY):
    # Authenticate with the server
    gc = girder_client.GirderClient(apiUrl=APIURL)
    gc.authenticate(apiKey=APIKEY)
    return gc

In [ ]:
conn = get_connection(APIURL, APIKEY)

In [ ]:
thumbnail_rgb = get_slide_thumbnail(conn, SAMPLE_SLIDE_ID)
(thumb_height, thumb_width, _) = np.shape(thumbnail_rgb)

plt.imshow(thumbnail_rgb)

In [ ]:
labeled, mask = get_tissue_mask(
        thumbnail_rgb, deconvolve_first=False,
        n_thresholding_steps=1, sigma=0.1, min_size=5)

In [ ]:
#Erosion, Opening(instead of erosion)
selem = disk(1)
# eroded = erosion(mask, selem)
# eroded_labelled = label(eroded, background = 0)


In [ ]:
opened = opening(mask, selem)

# assigning labels to disconnected labels of the mask
opened_label = label(opened, background = 0) 

In [ ]:
vals = np.random.rand(256,3)
vals[0, ...] = [0.9, 0.9, 0.9]
cMap = ListedColormap(1 - vals)

f, ax = plt.subplots(1, 3, figsize=(20, 20))
ax[0].imshow(thumbnail_rgb)
ax[1].imshow(labeled, cmap=cMap) # all tissue regions
ax[2].imshow(opened_label, cmap=cMap) # labels of largest tissue region
plt.show()

In [ ]:
props = regionprops_table(opened_label, properties=('area','centroid'))

In [ ]:
from scipy import ndimage as ndi
new_bin = remove_small_objects(opened_label, min_size = 5000)
filled_bin = ndi.binary_fill_holes(new_bin>0)

In [ ]:
f, ax = plt.subplots(figsize=(20,20))
ax.imshow(filled_bin, cmap=plt.cm.gray)
plt.show()

In [ ]:
img_tissue_boundary = segmentation.find_boundaries(filled_bin)
(tissue_x, tissue_y) = np.nonzero(img_tissue_boundary)

In [ ]:
area = pd.DataFrame(props)
#area.sort_values(axis=0, ascending=True, inplace=False )
area.sort_values("area", axis = 0, ascending = False,
                 inplace = True, na_position ='last')
area.head()